# Optic Diseases
https://www.kaggle.com/datasets/gunavenkatdoddi/eye-diseases-classification/data

In [29]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from tqdm import tqdm

%autosave 30

Autosaving every 30 seconds


## Dataset Exploration
* Identification of number of samples for each category
* Creation of a unified dataframe, containing paths and respective category

In [23]:
dataPath = r"C:\Users\asuto\Desktop\Work\Eye Diseases\dataset"
os.listdir(dataPath)

['cataract', 'diabetic_retinopathy', 'glaucoma', 'normal']

In [32]:
%%time
total = 0
for disease in os.listdir(dataPath):
    diseasePath = os.path.join(dataPath,disease)
    print(f"Number of samples for {disease} case: {len(os.listdir(diseasePath))}")
    total += len(os.listdir(diseasePath))
print(f"Total samples: {total}")

Number of samples for cataract case: 1038
Number of samples for diabetic_retinopathy case: 1098
Number of samples for glaucoma case: 1007
Number of samples for normal case: 1074
Total samples: 4217
CPU times: total: 46.9 ms
Wall time: 29.3 ms


In [42]:
%%time
diseaseMap = {"filepath":[],"label":[]}

for disease in tqdm(os.listdir(dataPath)):
    diseasePath = os.path.join(dataPath,disease)

    for path in os.listdir(diseasePath):
        diseaseMap['filepath'].append(path)
        diseaseMap['label'].append(disease)

opticDf = pd.DataFrame(diseaseMap).sample(frac=1)
opticDf.head()

100%|██████████| 4/4 [00:00<00:00, 617.22it/s]

CPU times: total: 15.6 ms
Wall time: 29.3 ms


,filepath,label
460,_13_3987009.jpg,cataract
1795,11182_right.jpeg,diabetic_retinopathy
1070,10046_left.jpeg,diabetic_retinopathy
1522,1075_left.jpeg,diabetic_retinopathy
2373,1486_left.jpg,glaucoma


In [43]:
opticDf.to_csv(r"C:\Users\asuto\Desktop\Work\Eye Diseases\opticDiseasesDataframe.csv",index=False)